# 演習環境用VCノードのスケールアウト

---

演習環境用のVCノードをスケールアウトさせます。

## 全体構成

![構成](images/guacamole-201-01.png)

# パラメータの設定

## UnitGroup名の指定

操作対象となる UnitGroup の名前を指定してください。

In [1]:
unit_group = 'Guacamole-C'

次のセルを実行すると「01-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、上のセルで指定した UnitGroup名に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [2]:
%run scripts/group.py
gvars = load_group_vars(unit_group)

>「01-パラメータの設定.ipynb」で group_vars ファイルに保存した値を読み込むために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。このスクリプトでは、指定された UnitGroup名に対応する YAMLファイルを `group_vars/` ディレクトリから読み込み、その値を Python の辞書型変数に格納しています。


## VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [3]:
from getpass import getpass
vcc_access_token = getpass()

········


入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [4]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

# 演習環境用VCノードのスケールアウト


## 現在の状態の確認

演習環境用に起動している VC ノードを一覧表示します。

In [5]:
ugroup = vcp.get_ugroup(unit_group)
unit = ugroup.get_unit('client')
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,49,Guacamole-C,client,RUNNING,1,bb3fa509...,RUNNING,172.30.2.167,i-0a0c61fa15a65c6e6,VCP-e84fbb63-88b49866,none


Grafanaの表示を確認します。次のセルを実行すると表示されるURLをアクセスしてください。

> ログイン画面が表示された場合は、ユーザ名/パスワードに admin/adminを入力してください。

In [6]:
print(f'https://{gvars["server_name"]}:{gvars["grafana_port"]}/grafana/d/handson/vcp-metrics?refresh=5s')

https://ocs1046.vcp-handson.org:8443/grafana/d/handson/vcp-metrics?refresh=5s


Grafanaに２つのVCノードに対応するグラフが表示されているはずです。もしGrafanaにVCノードに対応するグラフが表示されていない場合は次のセルを実行してからもう一度 Grafana の表示を確認してください。

In [7]:
!ansible {unit_group} -b -a 'systemctl restart cadvisor'

172.30.2.167 | SUCCESS | rc=0 >>


172.30.2.100 | SUCCESS | rc=0 >>




## VCノードのスケールアウト

演習環境用VCノードを１ノード追加してみます。

> ２分程度掛かります。

In [8]:
unit.add_nodes()

2019-03-20 19:59:05,495 - INFO - INDEFINITE ... 0 sec
2019-03-20 19:59:10,703 - INFO - INDEFINITE ... 5 sec
2019-03-20 19:59:15,906 - INFO - INDEFINITE ... 10 sec
2019-03-20 19:59:21,107 - INFO - INDEFINITE ... 15 sec
2019-03-20 19:59:26,313 - INFO - INDEFINITE ... 20 sec
2019-03-20 19:59:31,519 - INFO - INDEFINITE ... 25 sec
2019-03-20 19:59:36,726 - INFO - INDEFINITE ... 30 sec
2019-03-20 19:59:41,933 - INFO - INDEFINITE ... 35 sec
2019-03-20 19:59:47,134 - INFO - INDEFINITE ... 40 sec
2019-03-20 19:59:52,342 - INFO - INDEFINITE ... 45 sec
2019-03-20 19:59:57,553 - INFO - INDEFINITE ... 50 sec
2019-03-20 20:00:02,849 - INFO - INDEFINITE ... 55 sec
2019-03-20 20:00:08,247 - INFO - INDEFINITE ... 60 sec
2019-03-20 20:00:13,499 - INFO - INDEFINITE ... 65 sec
2019-03-20 20:00:18,733 - INFO - INDEFINITE ... 70 sec
2019-03-20 20:00:23,949 - INFO - INDEFINITE ... 75 sec
2019-03-20 20:00:29,153 - INFO - INDEFINITE ... 80 sec
2019-03-20 20:00:34,363 - INFO - INDEFINITE ... 85 sec
2019-03-20 2

複数のノードを追加する場合は、`add_nodes()` の引数`num_add_nodes`に追加するノード数を指定してください。例えば３ノードを追加する場合は、以下のようになります。

```
unit.add_nodes(num_add_nodes=3)
```

VC ノードを一覧表示してみます。

In [9]:
unit.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,49,Guacamole-C,client,RUNNING,1,bb3fa509...,RUNNING,172.30.2.167,i-0a0c61fa15a65c6e6,VCP-e84fbb63-88b49866,none
1,49,Guacamole-C,client,RUNNING,2,85decf6b...,RUNNING,172.30.2.232,i-0e78db5a658826681,VCP-e84fbb63-88b49866,none


## Ansibleの設定

Ansible のインベントリの記述を追加したVCノードについて更新を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [10]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

for addr in ugroup.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

# Host 172.30.2.100 found: line 40 type ECDSA
# Host 172.30.2.100 found: line 41 type RSA
# Host 172.30.2.100 found: line 42 type ED25519
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.100 SSH-2.0-OpenSSH_7.4
# 172.30.2.100 SSH-2.0-OpenSSH_7.4
# 172.30.2.100 SSH-2.0-OpenSSH_7.4
# Host 172.30.2.167 found: line 40 type ECDSA
# Host 172.30.2.167 found: line 41 type RSA
# Host 172.30.2.167 found: line 42 type ED25519
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.167 SSH-2.0-OpenSSH_7.4
# 172.30.2.167 SSH-2.0-OpenSSH_7.4
# 172.30.2.167 SSH-2.0-OpenSSH_7.4
/home/bit_kun/.ssh/known_hosts updated.
Original contents retained as /home/bit_kun/.ssh/known_hosts.old
# 172.30.2.232 SSH-2.0-OpenSSH_7.4
# 172.30.2.232 SSH-2.0-OpenSSH_7.4
# 172.30.2.232 SSH-2.0-OpenSSH_7.4


起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [11]:
import os

with open('hosts', 'w') as f:
    f.write(f'''
[{unit_group}:children]
{unit_group}-server
{unit_group}-client
    
[{unit_group}-server]
{os.linesep.join(ugroup.find_ip_addresses(unit_name='server'))}

[{unit_group}-client]
{os.linesep.join(ugroup.find_ip_addresses(unit_name='client'))}

[{unit_group}:vars]
ansible_ssh_private_key_file={gvars["ssh_private_key_path"]}
''')
    
!cat ./hosts


[Guacamole-C:children]
Guacamole-C-server
Guacamole-C-client
    
[Guacamole-C-server]
172.30.2.100

[Guacamole-C-client]
172.30.2.167
172.30.2.232

[Guacamole-C:vars]
ansible_ssh_private_key_file=/home/bit_kun/.ssh/id_rsa


UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [12]:
!ansible {unit_group} -m ping

172.30.2.100 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.30.2.167 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.30.2.232 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


## 追加したVCノードをGrafanaで確認する

Grafanaの表示を確認します。次のセルを実行すると表示されるURLをアクセスしてください。

In [13]:
print(f'https://{gvars["server_name"]}:{gvars["grafana_port"]}/grafana/d/handson/vcp-metrics?refresh=5s')

https://ocs1046.vcp-handson.org:8443/grafana/d/handson/vcp-metrics?refresh=5s


Grafanaに３つのVCノードに対応するグラフが表示されているはずです。もしGrafanaにVCノードに対応するグラフが表示されていない場合は次のセルを実行してからもう一度 Grafana の表示を確認してください。

In [14]:
!ansible {unit_group} -b -a 'systemctl restart cadvisor'

172.30.2.100 | SUCCESS | rc=0 >>


172.30.2.167 | SUCCESS | rc=0 >>


172.30.2.232 | SUCCESS | rc=0 >>




#  演習環境の利用

今回構築した環境では、演習環境コンテナに割り当てるリソース量を、１つのVCノードに１つのコンテナのみ実行できるような量を設定しています。そのため異なるユーザ名で演習環境にログインすると、それぞれ異なるVCノードで実行しているコンテナ環境を利用することになります。

このことをGrafana で確認しやすくするために、演習環境に対してCPU負荷をかけてみます。

まずは、ユーザ/パスワードに `user01`/`user01`を指定してGuacamoleにログインして、その後で演習環境にログインしてください。演習環境への接続は `SSH`/`RDP` のどちらで構いません。

演習環境にログインしたらターミナルで以下のコマンドを実行してください。

```
yes > /dev/null
```

![user01負荷](images/guacamole-203-02.png)

CPU負荷をかけた状態で Grafanaを確認してください。

In [15]:
print(f'https://{gvars["server_name"]}:{gvars["grafana_port"]}/grafana/d/handson/vcp-metrics?refresh=5s')

https://ocs1046.vcp-handson.org:8443/grafana/d/handson/vcp-metrics?refresh=5s


以下のキャプチャ画面に示すように[CPU Uage per Node]のグラフと[CPU Usage per Container]のグラフのうちのひとつで負荷が高くなっていることが確認できます。

![user01負荷grafana](images/guacamole-203-03.png)

次に`user01`をログアウトしてから、`user02`で演習環境にログインしてください。その後、先ほどと同じようにターミナルで

```
yes > /dev/null
```

を実行してCPU負荷をかけてください。

![user02負荷](images/guacamole-203-04.png)

この状態で Grafanaを確認してください。以下のキャプチャ画面に示すように[CPU Uage per Node]のグラフと[CPU Usage per Container]のグラフで先ほどとは異なるVCノードとコンテナの負荷が高くなっていることが確認できます。

![user02負荷grafana](images/guacamole-203-05.png)